In [ ]:
!wget https://github.com/maxpumperla/dl4j_coursera/releases/download/v0.4/dl4j-snapshot.jar

In [ ]:
!wget https://raw.githubusercontent.com/maxpumperla/dl4j_coursera/master/iris.txt

In [ ]:
import numpy
import pandas
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

In [ ]:
# fix random seed for reproducibility
seed = 10
numpy.random.seed(seed)

In [ ]:
# load dataset
dataframe = pandas.read_csv("iris.txt", header=None)
dataset = dataframe.values
X = dataset[:, 0:4].astype(float)
Y = dataset[:, 4]

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
model = Sequential()
model.add(Dense(4, input_dim=4, activation="relu"))
model.add(Dense(3, activation="sigmoid"))
# Compile model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X, dummy_y, epochs=20, batch_size=5)
model.save("iris_model.h5")

In [ ]:
# some learners constantly reported 502 errors in Watson Studio.
# This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
# This is a workaround to limit resource consumption

from keras import backend as K


K.set_session(
    K.tf.Session(
        config=K.tf.ConfigProto(
            intra_op_parallelism_threads=1, inter_op_parallelism_threads=1
        )
    )
)

In [ ]:
!$SPARK_HOME/bin/spark-submit \
--class skymind.dsx.KerasImportCSVSparkRunner \
--files iris.txt,iris_model.h5 \
--master $MASTER \
dl4j-snapshot.jar \
-batchSizePerWorker 15 \
-indexLabel 4

In [ ]:
!mv dl4j-snapshot.jar.1 dl4-snapshot.jar

In [ ]:
!ls dl4j-snapshot.jar